# Wing Kinematics Extraction from NJVID Insect Flight Videos

> This notebook demonstrates the full tracking and analysis pipeline using data from Professor Mitsunori Denda's NJVID Insect Flight Collection.

**Pipeline:**
1. Load video / generate synthetic data
2. Track body centroid and wing tip
3. Compute angular displacement, velocity, acceleration
4. Estimate wingbeat frequency (FFT + peak detection)
5. Export to CSV and generate publication-quality plots
6. Generate 3D parametric wing model

In [ ]:
import sys
sys.path.insert(0, '..')

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from src.analysis import compute_kinematics, smooth_signal, differentiate, compute_frequency
from src.export import to_csv, generate_summary
from src.visualization import (
    plot_trajectory, plot_angle_timeseries,
    plot_velocity_acceleration, plot_frequency_spectrum,
    plot_dashboard
)
from run_tracker import generate_synthetic_data

print('All modules loaded successfully.')

## 1. Generate Synthetic Tracking Data

We simulate a butterfly with a 12 Hz wingbeat that includes harmonics (not a pure sine wave), mimicking the nonlinear characteristics of real insect wing motion.

To use real video data, replace this cell with:
```python
from src.tracker import run_tracking
results = run_tracking('data/raw/morpho_peleides.mp4', species='Morpho peleides')
```

In [ ]:
species = 'Morpho peleides'
results = generate_synthetic_data(fps=120.0, duration=2.0, freq=12.0, species=species)

print(f"Species: {species}")
print(f"Frames tracked: {len(results['frames'])}")
print(f"FPS: {results['metadata']['fps']}")
print(f"Duration: {results['metadata']['duration_s']:.2f} s")
print(f"\nFirst 5 angles (rad): {results['angles'][:5]}")

## 2. Compute Kinematics

The kinematic pipeline:
1. **Savitzky-Golay smoothing** — preserves waveform shape while reducing measurement noise
2. **Central difference differentiation** — computes angular velocity (rad/s) and angular acceleration (rad/s²)
3. **FFT frequency estimation** — identifies dominant wingbeat frequency
4. **Peak-based frequency estimation** — complementary time-domain method

In [ ]:
kinematics = compute_kinematics(
    results['angles'],
    results['metadata']['fps'],
    smooth_window=11,
    smooth_poly=3
)

print(generate_summary(kinematics, species))

## 3. Visualizations

### 3.1 Flight Trajectory
Body centroid and wing tip positions in pixel coordinates. Color gradient shows time progression.

In [ ]:
plot_trajectory(results['centroids'], results['wing_tips'], species=species)

### 3.2 Angular Displacement Over Time
Raw tracking data (gray) vs Savitzky-Golay smoothed signal (blue). The smoothing preserves peak locations and waveform asymmetry.

In [ ]:
plot_angle_timeseries(
    kinematics['time'],
    kinematics['angles_raw'],
    kinematics['angles_smoothed'],
    species=species
)

### 3.3 Angular Velocity & Acceleration
Computed via central difference numerical differentiation. Velocity peaks at the midstroke; acceleration peaks at stroke reversal.

In [ ]:
plot_velocity_acceleration(
    kinematics['time'],
    kinematics['angular_velocity'],
    kinematics['angular_acceleration'],
    species=species
)

### 3.4 Wingbeat Frequency Spectrum
FFT magnitude spectrum. The dominant peak corresponds to the wingbeat frequency. Note the presence of harmonics — evidence that the motion is not purely sinusoidal.

In [ ]:
plot_frequency_spectrum(
    kinematics['fft_freqs'],
    kinematics['fft_magnitudes'],
    kinematics['wingbeat_freq_fft'],
    species=species
)

### 3.5 Full Dashboard

In [ ]:
plot_dashboard(kinematics, results, species=species)

## 4. Export to CSV

The CSV includes metadata comments and columns for frame index, time, raw and smoothed angles, angular velocity, angular acceleration, and tracked positions.

In [ ]:
import os
os.makedirs('../output/csv', exist_ok=True)

csv_path = to_csv(
    kinematics, results,
    '../output/csv/morpho_peleides_kinematics.csv',
    species=species
)

# Show first few lines of the CSV
with open(csv_path) as f:
    for i, line in enumerate(f):
        print(line.rstrip())
        if i > 15:
            print('...')
            break

## 5. Numerical Methods Detail

### 5.1 Central Difference vs Forward Difference
We use central differences because they have O(h²) truncation error, compared to O(h) for forward differences.

In [ ]:
# Demonstrate central difference accuracy on a known function
dt = 1.0 / 120.0
t = np.arange(0, 1, dt)
y = np.sin(2 * np.pi * 10 * t)
y_true_deriv = 2 * np.pi * 10 * np.cos(2 * np.pi * 10 * t)

# Central difference
y_central = differentiate(y, dt)

# Forward difference (for comparison)
y_forward = np.zeros_like(y)
y_forward[:-1] = (y[1:] - y[:-1]) / dt
y_forward[-1] = y_forward[-2]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(t[:50], y_true_deriv[:50], 'k-', label='Analytical', linewidth=2)
ax1.plot(t[:50], y_central[:50], 'b--', label='Central diff', linewidth=1.5)
ax1.plot(t[:50], y_forward[:50], 'r:', label='Forward diff', linewidth=1.5)
ax1.set_xlabel('Time (s)')
ax1.set_ylabel('Derivative')
ax1.set_title('Derivative Comparison')
ax1.legend()

ax2.semilogy(t[5:-5], np.abs(y_central[5:-5] - y_true_deriv[5:-5]), 'b-', label='Central diff error')
ax2.semilogy(t[5:-5], np.abs(y_forward[5:-5] - y_true_deriv[5:-5]), 'r-', label='Forward diff error')
ax2.set_xlabel('Time (s)')
ax2.set_ylabel('Absolute Error')
ax2.set_title('Error Comparison')
ax2.legend()

plt.tight_layout()
plt.show()

print(f'Central diff RMSE: {np.sqrt(np.mean((y_central[5:-5] - y_true_deriv[5:-5])**2)):.6f}')
print(f'Forward diff RMSE: {np.sqrt(np.mean((y_forward[5:-5] - y_true_deriv[5:-5])**2)):.6f}')

## 6. 3D Wing Model

Generate a parametric 3D model of the Morpho peleides wing based on morphological parameters.

In [ ]:
from src.wing_model import (
    SPECIES_PARAMS, generate_wing_surface,
    generate_wing_profile, generate_full_model
)

params = SPECIES_PARAMS['Morpho peleides']
print(f"Wing Parameters for {species}:")
for k, v in params.items():
    print(f"  {k}: {v}")

# Visualize wing profile at root and tip
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

root_profile = generate_wing_profile(
    params['root_chord_mm'], params['camber_fraction'], params['camber_position']
)
tip_profile = generate_wing_profile(
    params['tip_chord_mm'], params['camber_fraction'] * 0.5, params['camber_position']
)

ax1.plot(root_profile[:, 0], root_profile[:, 1], 'b-', linewidth=2)
ax1.fill_between(root_profile[:, 0], root_profile[:, 1], alpha=0.1, color='blue')
ax1.set_xlabel('Chord (mm)')
ax1.set_ylabel('Camber (mm)')
ax1.set_title('Root Wing Profile')
ax1.set_aspect('equal')
ax1.grid(True, alpha=0.3)

ax2.plot(tip_profile[:, 0], tip_profile[:, 1], 'r-', linewidth=2)
ax2.fill_between(tip_profile[:, 0], tip_profile[:, 1], alpha=0.1, color='red')
ax2.set_xlabel('Chord (mm)')
ax2.set_ylabel('Camber (mm)')
ax2.set_title('Tip Wing Profile')
ax2.set_aspect('equal')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# 3D wing surface visualization
from mpl_toolkits.mplot3d import Axes3D

X, Y, Z = generate_wing_surface(params, n_span=40, n_chord=30)

fig = plt.figure(figsize=(14, 8))
ax = fig.add_subplot(111, projection='3d')

# Right wing
ax.plot_surface(X, Y, Z, alpha=0.7, cmap='Blues', edgecolor='none')
# Left wing (mirrored)
ax.plot_surface(X, -Y, Z, alpha=0.7, cmap='Blues', edgecolor='none')

ax.set_xlabel('X (mm)')
ax.set_ylabel('Y (mm) — Span')
ax.set_zlabel('Z (mm) — Camber')
ax.set_title(f'{species} — Parametric Wing Model')
ax.view_init(elev=25, azim=-60)

plt.tight_layout()
plt.show()

In [ ]:
# Generate and save STL file
import os
os.makedirs('../output/models', exist_ok=True)

stl_path = generate_full_model('Morpho peleides', flap_angle_deg=0.0,
                                output_dir='../output/models/')
print(f'\nSTL file size: {os.path.getsize(stl_path) / 1024:.1f} KB')

## Summary

This notebook demonstrated:
1. **Tracking pipeline** — body centroid and wing tip extraction
2. **Signal processing** — Savitzky-Golay smoothing with phase preservation
3. **Numerical differentiation** — Central difference with O(h²) accuracy
4. **Frequency analysis** — FFT-based wingbeat frequency estimation
5. **3D modeling** — Parametric wing geometry based on species morphology
6. **CSV export** — Publication-ready data with metadata

The exported CSV can be used as input for the companion Julia project (`nonlinear-wing-model-julia`) for nonlinear dynamical modeling.